# Импорт библиотек и функций 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import warnings
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot, plot_forecast_component
from fbprophet.plot import plot_yearly
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



# Подготовка данных

Для анализа предлагаются ежедневные данные о коммулятивных подтвержденных случаях и смертности в период с 22-го января 2020 года по 15-ое мая 2020 года для 184 регионов. Прогнозирование будет производится на датах со 1-го мая 2020 года по 15-ое мая 2020 года. Будут проанализированы временные ряды для общемировых (суммарных) показателей, а также частные данный по Российской Федерации. Далее в отдельные датафреймы выделяются данные по России и агрегированные данные по миру. Для наглядности приведены временные зависимости.

In [ ]:
data = pd.read_csv("train.csv")
data['Date'] = data['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
data_russia = data[data.Country_Region == "Russia"][["Date", "ConfirmedCases", "Fatalities"]].set_index("Date")
data_world = data.groupby("Date", as_index=False).agg("sum")[["Date", "ConfirmedCases", "Fatalities"]].set_index("Date")

In [ ]:
iplot([go.Scatter(x=data_world.index, y=data_world.ConfirmedCases, name='Кол-во подтвержденных случаев по миру'),\
       go.Scatter(x=data_russia.index, y=data_russia.ConfirmedCases, name='Кол-во подтвержденных случаев по России')])

In [ ]:
iplot([go.Scatter(x=data_world.index, y=data_world.Fatalities, name='Кол-во фатальных исходов по миру'),\
       go.Scatter(x=data_russia.index, y=data_russia.Fatalities, name='Кол-во фатальных исходов по России')])

# Полиномиальная интерполяция (выделение тренда)

Для нахождения аппроксимирующего полинома будем использовать метод np.polyfit. Он подбирает полиномиальные коэффициенты методом наименьших квадратов (МНК). Методу np.polyfit необходимо сообщать степень аппроксимирующего полинома, поэтому реализуем функцию, которая будет возвращать степень, минимизирующую метрику MAE. 

In [ ]:
def optimal_degree(x, y, min_degree=1, max_degree=100):
  x = np.arange(len(x)).reshape(len(x), 1)
  y = np.array(y)
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
  min_mae = np.inf
  deg_mae = 0
  for i in range(min_degree, max_degree + 1):

    # Train features
    poly_features = PolynomialFeatures(degree=i, include_bias=False)
    x_poly_train = poly_features.fit_transform(x_train)

    # Linear regression
    poly_reg = LinearRegression()
    poly_reg.fit(x_poly_train, y_train)

    # Compare with test data
    x_poly_test = poly_features.fit_transform(x_test)
    poly_predict = poly_reg.predict(x_poly_test)

    poly_mae = mean_absolute_error(y_test, poly_predict) 
    if poly_mae < min_mae:
      min_mae = poly_mae
      deg_mae = i
  
  return deg_mae

Напишем также функцию, которая будет возвращать аппроксимирующий полином. 

In [ ]:
def polynomial_interpolation(x, y):
  deg = optimal_degree(x, y)
  model = np.polyfit(x=np.arange(len(x)), y=np.array(y), deg=deg)
  polynom = np.poly1d(model)
  return polynom

И получим результаты для каждого из временных рядов.

In [ ]:
pol_russia_conf = polynomial_interpolation(data_russia.index, data_russia.ConfirmedCases)(np.arange(len(data_russia)))
pol_russia_fatal = polynomial_interpolation(data_russia.index, data_russia.Fatalities)(np.arange(len(data_russia)))
pol_world_conf = polynomial_interpolation(data_world.index, data_world.ConfirmedCases)(np.arange(len(data_world)))
pol_world_fatal = polynomial_interpolation(data_world.index, data_world.Fatalities)(np.arange(len(data_world)))

Для каждого из рядов построим графики, отображающие реальные и аппроксимированные данные.

**Кол-во подтвержденных случаев по миру**

In [ ]:
iplot([go.Scatter(x=data_world.index, y=data_world.ConfirmedCases, name='Оригинальные данные'),\
       go.Scatter(x=data_world.index, y=pol_world_conf, name='Аппроксимация')])

**Кол-во фатальных исходов по миру**

In [ ]:
iplot([go.Scatter(x=data_world.index, y=data_world.Fatalities, name='Оригинальные данные'),\
       go.Scatter(x=data_world.index, y=pol_world_fatal, name='Аппроксимация')])

In [ ]:
iplot([go.Scatter(x=data_russia.index, y=data_russia.ConfirmedCases, name='Оригинальные данные'),\
       go.Scatter(x=data_russia.index, y=pol_russia_conf, name='Аппроксимация')])

**Кол-во фатальных исходов по России**

In [ ]:
iplot([go.Scatter(x=data_russia.index, y=data_russia.Fatalities, name='Оригинальные данные'),\
       go.Scatter(x=data_russia.index, y=pol_russia_fatal, name='Аппроксимация')])

# Эктраполяция (прогнозирвоание) с помощью ARIMA модели

Вычтем из оригинальных временных зависимостей полученные тренды и для каждой из временных зависимостей определим порядок разности, на которых достигается стационарность ряда. Будем проверять данные на стационарность с помощью теста Дики-Фуллера.

In [ ]:
trend_r_c = data_russia.ConfirmedCases - pol_russia_conf
trend_r_f = data_russia.Fatalities - pol_russia_fatal
trend_w_c = data_world.ConfirmedCases - pol_world_conf
trend_w_f = data_world.Fatalities - pol_world_fatal

Объявим фунцию, которая будет определять стационарность ряда:

In [ ]:
def is_stationary(series, name):
  test = sm.tsa.stattools.adfuller(series)
  return name + " is stationary" if test[0] < test[4]['5%'] else name + " is not stationary"

И проверим ряды на стационарность

In [ ]:
print(is_stationary(trend_r_c, "trend_r_c"))
print(is_stationary(trend_r_f, "trend_r_f"))
print(is_stationary(trend_w_c, "trend_w_c"))
print(is_stationary(trend_w_f, "trend_w_f"))

trend_r_c is stationary
trend_r_f is stationary
trend_w_c is stationary
trend_w_f is stationary


Как видно, все данные уже являются стационарными, поэтому в качестве параметра d в ARIMA будем использовать d = 0.

Определим функцию, которая будет производить прогнозирование ряда методом ARIMA:

In [ ]:
def arima(series, p, d, q):
  model = sm.tsa.ARIMA(series[:"2020-05-01"], (p,d,q))
  model = model.fit()
  return model.predict(start='2020-01-22', end='2020-05-15')


И применим ее для каждого из рядов

**Кол-во подтвержденных случаев по миру**

In [ ]:
w_c_forecast = arima(trend_w_c, 6, 0, 2)
iplot([go.Scatter(x=data_world.index, y=trend_w_c, name='Оригинальные данные за вычетом тренда'),\
       go.Scatter(x=data_world.index, y=list(w_c_forecast), name='Прогноз')])

**Кол-во фатальных исходов по миру**

In [ ]:
w_f_forecast = arima(trend_w_f, 10, 0, 0)
iplot([go.Scatter(x=data_world.index, y=trend_w_f, name='Оригинальные данные за вычетом тренда'),\
       go.Scatter(x=data_world.index, y=list(w_f_forecast), name='Прогноз')])

**Кол-во подтвержденных случаев по России**

In [ ]:
r_c_forecast = arima(trend_r_c, 3, 0, 1)
iplot([go.Scatter(x=data_world.index, y=trend_r_c, name='Оригинальные данные за вычетом тренда'),\
       go.Scatter(x=data_world.index, y=list(r_c_forecast), name='Прогноз')])

**Кол-во фатальных исходов по России**

In [ ]:
r_f_forecast = arima(trend_r_f, 3, 0, 1)
iplot([go.Scatter(x=data_world.index, y=trend_r_f, name='Оригинальные данные за вычетом тренда'),\
       go.Scatter(x=data_world.index, y=list(r_f_forecast), name='Прогноз')])

Найдем также значения метрик MAE и MAPE для каждого из рядов:

In [ ]:
print("MAE(w_c_forecast, trend_w_c) = ", mean_absolute_error(w_c_forecast, trend_w_c), "MAPE(w_c_forecast, trend_w_c) = ", mean_absolute_percentage_error(w_c_forecast, trend_w_c))
print("MAE(w_f_forecast, trend_w_f) = ", mean_absolute_error(w_f_forecast, trend_w_f), "MAPE(w_f_forecast, trend_w_f) = ", mean_absolute_percentage_error(w_f_forecast, trend_w_f))
print("MAE(r_c_forecast, trend_r_c) = ", mean_absolute_error(r_c_forecast, trend_r_c), "MAPE(r_c_forecast, trend_r_c) = ", mean_absolute_percentage_error(r_c_forecast, trend_r_c))
print("MAE(r_f_forecast, trend_r_f) = ", mean_absolute_error(r_f_forecast, trend_r_f), "MAPE(r_f_forecast, trend_r_f) = ", mean_absolute_percentage_error(r_f_forecast, trend_r_f))

MAE(w_c_forecast, trend_w_c) =  3831.027195150717 MAPE(w_c_forecast, trend_w_c) =  4.572605785322481
MAE(w_f_forecast, trend_w_f) =  274.7304703527424 MAPE(w_f_forecast, trend_w_f) =  3.245904120117835
MAE(r_c_forecast, trend_r_c) =  124.9510655390178 MAPE(r_c_forecast, trend_r_c) =  13.685317018834288
MAE(r_f_forecast, trend_r_f) =  3.580989243659014 MAPE(r_f_forecast, trend_r_f) =  1.811330601169725


# Прогнозирование с помощью модели и библиотеки Prophet 

Переведем все данные в необходимый для работы с библиотекой вид

In [ ]:
wc = pd.DataFrame({"ds": trend_w_c.index, "y": trend_w_c.values})
wf = pd.DataFrame({"ds": trend_w_f.index, "y": trend_w_f.values})
rc = pd.DataFrame({"ds": trend_r_c.index, "y": trend_r_c.values})
rf = pd.DataFrame({"ds": trend_r_f.index, "y": trend_r_f.values})

Определим функцию, которая будет прогнозировать значения ряда методом Prophet

In [ ]:
def prophet_pred(series, period):
  m = Prophet()
  m.fit(series[series.ds <= "2020-05-01"])
  future = m.make_future_dataframe(periods=period , freq = 'd')
  return m.predict(future)

И применим ее для каждого из рядов

In [ ]:
wc_prophet = prophet_pred(wc, 14)
wf_prophet = prophet_pred(wf, 14)
rc_prophet = prophet_pred(rc, 14)
rf_prophet = prophet_pred(rf, 14)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Построим графики

**Кол-во подтвержденных случаев по миру**

In [ ]:
iplot([go.Scatter(x=trend_w_c.index, y=trend_w_c, name='fact'),\
       go.Scatter(x=wc_prophet.ds, y=wc_prophet.yhat, name='forecast')])

**Кол-во фатальных исходов по миру**

In [ ]:
iplot([go.Scatter(x=trend_w_f.index, y=trend_w_f, name='fact'),\
       go.Scatter(x=wf_prophet.ds, y=wf_prophet.yhat, name='forecast')])

**Кол-во подтвержденных случаев по России**

In [ ]:
iplot([go.Scatter(x=trend_r_c.index, y=trend_r_c, name='fact'),\
       go.Scatter(x=rc_prophet.ds, y=rc_prophet.yhat, name='forecast')])

**Кол-во фатальных исходов по России**

In [ ]:
iplot([go.Scatter(x=trend_r_f.index, y=trend_r_f, name='fact'),\
       go.Scatter(x=rf_prophet.ds, y=rf_prophet.yhat, name='forecast')])

И вычислим метрики MAE и MAPE

In [ ]:
print("MAE(wc_prophet.yhat, trend_w_c) = ", mean_absolute_error(wc_prophet.yhat, trend_w_c), "MAPE(wc_prophet.yhat, trend_w_c) = ", mean_absolute_percentage_error(wc_prophet.yhat, trend_w_c))
print("MAE(wf_prophet.yhat, trend_w_f) = ", mean_absolute_error(wf_prophet.yhat, trend_w_f), "MAPE(wf_prophet.yhat, trend_w_f) = ", mean_absolute_percentage_error(wf_prophet.yhat, trend_w_f))
print("MAE(rc_prophet.yhat, trend_r_c) = ", mean_absolute_error(rc_prophet.yhat, trend_r_c), "MAPE(rc_prophet.yhat, trend_r_c) = ", mean_absolute_percentage_error(rc_prophet.yhat, trend_r_c))
print("MAE(rf_prophet.yhat, trend_r_f) = ", mean_absolute_error(rf_prophet.yhat, trend_r_f), "MAPE(rf_prophet.yhat, trend_r_f) = ", mean_absolute_percentage_error(rf_prophet.yhat, trend_r_f))

MAE(wc_prophet.yhat, trend_w_c) =  8209.481023558554 MAPE(wc_prophet.yhat, trend_w_c) =  15.167531908423665
MAE(wf_prophet.yhat, trend_w_f) =  534.6670031836043 MAPE(wf_prophet.yhat, trend_w_f) =  2.207946205716433
MAE(rc_prophet.yhat, trend_r_c) =  157.68634010761775 MAPE(rc_prophet.yhat, trend_r_c) =  4.239455061850468
MAE(rf_prophet.yhat, trend_r_f) =  3.71614204035101 MAPE(rf_prophet.yhat, trend_r_f) =  54.85730394226143
